# Introdução
Este nottebook é destinado a análises de textos de reclamações de empresas de ecommerce. Para isso, são carregados os dados de abreviatura dos estados do Brasil, dados sobre o numero de clientes de cada empresa analisada, os dados das reclamações extraidas do reclame aqui  e os aquivos necessários para o préprocessamento. 

Os dados são ajustados e manipulados e algumas estatististica basiscas são realizadas e mostrada através de gráficos. Por fim, os dados são preprocesados e são realizadas a modelagem de tópico para em todos os dados e nos dados divididos em regiões. 

![Reclame Aqui](https://www.reclameaqui.com.br/_nuxt/assets/imgs/reclame-aqui-logo.svg) 


# 1. Importação da bibliotecas necessárias 

In [1]:
import pandas as pd 
import csv
import os
import numpy as np

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from time import time
import re
import os
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import nltk 
from nltk.stem import RSLPStemmer




import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ecommerce-consumidor-gov/submarino.csv
/kaggle/input/ecommerce-consumidor-gov/americanas.csv
/kaggle/input/ecommerce-consumidor-gov/magazineluiza.csv
/kaggle/input/ecommerce-consumidor-gov/mercado_livre.csv
/kaggle/input/ecommerce-consumidor-gov/netshoes.csv
/kaggle/input/preprocessamento/stopwords_pt_nltk.txt
/kaggle/input/preprocessamento/estados_brasil.csv
/kaggle/input/preprocessamento/abreviacao.csv
/kaggle/input/preprocessamento/regionalismos.csv
/kaggle/input/preprocessamento/letras.csv
/kaggle/input/preprocessamento/LIWC2007_Portugues_win.dic.txt


# 2. Carregamento dos dados 

## 2.1 Carregar nomes e abreviatura dos estados do Brasil 

In [2]:
caminho_estados = "/essenciais/"
# carregar estados do Brasil
df_estados = pd.read_csv(caminho_estados+"estados_brasil.csv", sep=",",engine="python", error_bad_lines=False)

print("Dados com as abreviaturas foi carregado com sucesso!")

Dados com as abreviaturas foi carregado com sucesso!


## 2.2 Carregar número de clientes por empresa

In [3]:
# caminho_numero_clientes = "/kaggle/input/preprocessamento/"
# # carregar dados do numero de clientes providos pela anatel 
# df_clientes_por_companhia = pd.read_csv(caminho_numero_clientes+"acessos_corrigidos.csv", sep=";",engine="python", error_bad_lines=False)
# # df_clientes_states=df_clientes_states[(df_clientes_states.tecnologia == 'GSM') | (df_clientes_states.tecnologia == 'LTE') 
# #                                       | (df_clientes_states.tecnologia ==  'WCDMA')]
# print("Dados com o número de clientes foi carregado com sucesso!")

## 2.3 Carregar dados das reclamações 

### 2.3.1 Carregar dados reclame aqui

In [4]:
caminho_reclamacoes = "../dados_coletados/consumidor_gov/"
# caminho_reclamacoes = "../../datasets/reclame_aqui/"
df_americanas_com_loja_online = pd.read_csv(caminho_reclamacoes+"americanas.csv", sep="\t",engine="python", error_bad_lines=False)
df_magazine_luiza_loja_online = pd.read_csv(caminho_reclamacoes+"magazineluiza.csv", sep="\t",engine="python", error_bad_lines=False)
df_mercado_livre = pd.read_csv(caminho_reclamacoes+"mercado_livre.csv", sep="\t",engine="python", error_bad_lines=False)
df_netshoes = pd.read_csv(caminho_reclamacoes+"netshoes.csv", sep="\t",engine="python", error_bad_lines=False)
df_submarino = pd.read_csv(caminho_reclamacoes+"submarino.csv", sep="\t",engine="python", error_bad_lines=False)

print("\n\nDados do Reclame aqui das companhias de telecomunicações carregados com suscesso!")

print("\nQuantidade de reclamações")
print("americanas_com_loja_online - "+str(len(df_americanas_com_loja_online))+" reclamações")
print("magazine-luiza-loja-online - "+str(len(df_magazine_luiza_loja_online))+" reclamações")
print("mercado-livre - "+str(len(df_mercado_livre))+" reclamações")
print("netshoes - "+str(len(df_netshoes))+" reclamações")
print("submarino - "+str(len(df_submarino))+" reclamações")

Skipping line 1740: '	' expected after '"'
Skipping line 2781: '	' expected after '"'
Skipping line 3220: '	' expected after '"'
Skipping line 3898: '	' expected after '"'
Skipping line 12109: '	' expected after '"'
Skipping line 12217: '	' expected after '"'
Skipping line 12952: '	' expected after '"'
Skipping line 13638: '	' expected after '"'
Skipping line 13890: '	' expected after '"'
Skipping line 14755: '	' expected after '"'
Skipping line 2273: '	' expected after '"'
Skipping line 2970: '	' expected after '"'
Skipping line 5042: '	' expected after '"'
Skipping line 5532: '	' expected after '"'
Skipping line 5533: '	' expected after '"'
Skipping line 9245: '	' expected after '"'
Skipping line 21966: '	' expected after '"'
Skipping line 22402: '	' expected after '"'
Skipping line 25798: '	' expected after '"'
Skipping line 944: '	' expected after '"'




Dados do Reclame aqui das companhias de telecomunicações carregados com suscesso!

Quantidade de reclamações
americanas_com_loja_online - 16456 reclamações
magazine-luiza-loja-online - 5694 reclamações
mercado-livre - 26346 reclamações
netshoes - 4377 reclamações
submarino - 5169 reclamações


Skipping line 1893: '	' expected after '"'
Skipping line 3394: '	' expected after '"'


In [5]:
df_americanas_com_loja_online.head(2)

,empresa,status,data,local,relato,tempo_resposta,resposta,nota,avaliacao
0,Americanas.com,Resolvido,06/02/2019,Miracema do Tocantins - TO,"Efetuei o pagamento do pedido XXXXX-X86685281,...",(10 dia(s) depois),Olá Amanda Gomes Rocha!Devido ao prazo estipul...,NaN,<não há comentários do consumidor>
1,Americanas.com,Resolvido,11/02/2019,Altônia - PR,Cliente informa que fez compra na Lojas Americ...,(10 dia(s) depois),Olá Elizeu!Em atendimento à demanda registrada...,NaN,<não há comentários do consumidor>


## 2.4 Carregar dados para o Pré-processamento 

In [6]:
# Carregamento de dados para processar abreviação
dicionario_dados_abreviacao = {}
with open('/essenciais/abreviacao.csv',  encoding="utf-8-sig") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        dicionario_dados_abreviacao[row[0]]= row[1]
        
        
# carregando alfabeto
lista_dados_letras_alfabeto = []
with open('/essenciais/letras.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        lista_dados_letras_alfabeto.append(row[0])
        
# carregando stopwords
stop_words = []
with open('/essenciais/stopwords_pt_nltk.txt') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        stop_words.append(row[0])


 # carregando regionalismos
lista_dados_regionalismos = []
with open('/essenciais/regionalismos.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        lista_dados_regionalismos.append(row[0])
        
print("Carregamento dos dados para o pre-processamento finalizado! ")

Carregamento dos dados para o pre-processamento finalizado! 


# 3. Métodos 
Nesta subseção são definidos os metodos usados no decorrer do notebook

 ## 3.1 Metodo para a separação
 Este metodo pode ser usado para separa dados de uma coluna, de acordo com os parametros passados. 
Ele será usado para separar a coluna "Created" em data e hora, e a coluna "city_state" e cidade e estado.

In [7]:
def separa_coluna(data, spli):
    first = []
    second = []
    for da in data:
        temp = da.split(spli)
        first.append(temp[0])
        second.append(temp[1])
    return first, second

print("Metodo para a separação dos dados foi carregados com sucesso")

Metodo para a separação dos dados foi carregados com sucesso


## 3.2 Métodos para po préprocessamento

In [8]:
# Remover abrevoações
abbreviations_regex = r"|".join([r"\b" + key + r"\b" for key in dicionario_dados_abreviacao.keys()])
def transformar_abreviacao_em_palavras( text):
    matches = re.findall(abbreviations_regex, text)
    for match in matches:
        if match in dicionario_dados_abreviacao.keys():
            match_regex = r"\b" + match + r"\b"
            replacement = dicionario_dados_abreviacao[match]
            text = re.sub(match_regex, replacement, text)

    return text

# remover repetição de letras
def remover_repeticao_de_letras( text):

    for match in lista_dados_letras_alfabeto:
        match_regex = match + r"a{3,}|b{3,}|c{3,}|d{3,}|e{3,}|f{3,}|g{3,}|h{3,}|i{3,}|j{3,}|k{3,k}|l{3,}|m{3,}|n{3,}|o{3,}|p{3,}|q{3,}|r{3,}|s{3,}|t{3,}|u{3,}|v{3,}|w{3,}|x{3,}|y{3,}|z{3,}"
        replacement = match
        text = re.sub(match_regex, replacement, text)

    return text

# remover stopwords
def remover_stopwords(texts):
    a=[]
    for word in word_tokenize(str(texts)):
        if word not in stop_words:
            a.append(word)
    
    return a 

# remover regionalismo
def remover_regionalismo(text):
    for aa in lista_dados_regionalismos:
        text = re.sub(r"\b"+aa+r"\b", '', str(text))
    return text

# remover saudações 
def remover_saudacao(text):
    for aa in ["ola","oi","bom dia","boa tarde","boa noite"]:
        text = re.sub(r"\b"+aa+r"\b", '', str(text))

    return text
# remover acentuação
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

# fazer tokenização
def Tokenize(sentence):
    sentence = sentence.lower()
    sentence = nltk.word_tokenize(sentence)
    return sentence

# stemming das palavras
def Stemming(sentence):
    sentence = Tokenize(sentence)
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return u" ".join(word for word in phrase)

# remover nome empresa 
def remover_empresa_nome(text):
    empresas = [
        "mercadolivre",
        "mercado-livre",
        "mercado_livre",
        "submarino",
        "magazineluiza.com",
        "magazineluiza",
        "americanas.com", 
        "americanas", 
        "netshoes",
    ]
    for aa in empresas:
        text = re.sub(r"\b"+aa+r"\b", '', str(text))

    return text

# Metodo para executar o preporcessamento 
def processar_dados(data):
    
     # remove stopwords
    data = [' '.join(remover_stopwords(sent)) for sent in data]
    
    # lowerCase
    data = [sent.lower() for sent in data]
    
    #remove abreviacao
    data = [transformar_abreviacao_em_palavras(sent) for sent in data]

    #remove BRs
    data = [re.sub(r"<br \/>", ' ', sent) for sent in data]
    
     # Remove URLs
    data = [re.sub("http://\S+\s*", "", sent) for sent in data]
    data = [re.sub("https://\S+\s*", "", sent) for sent in data]
    data = [re.sub("www.\S+\s*", "", sent) for sent in data]
    
    # Acentuação 
    data = [ str(sent).encode('latin-1', 'ignore').decode('latin-1') for sent in data]

    # Transformar palavras para minusculo 
    data = [sent.lower() for sent in data]

    # remove tags
    data = [re.sub('\<script.*script>', '', str(sent)) for sent in data]

    data = [re.sub('\<a.*a>', '', str(sent)) for sent in data]

    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

    # remove HashTag
    data = [re.sub(r"#\w*", "", sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    #Remove RTs
    data = [re.sub("RT", "", sent) for sent in data]

    # Remove numbers 
    data = [re.sub(r"[0-9]+", "", sent) for sent in data]

    # Remove ponctuation 
    data = [re.sub(r"\.|,|;|!|\?|\ - |:", " ", sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # remove acentuation
    data =  [remove_accents(sent) for sent in data]

    # removeNoise
    data = [re.sub(r"\\|\.|(?<!\w)\w(?!\w)|>|<|,|\`|;|:|\+|-|_|\*|@|#|\$|%|&|\(|\)|\[|\]|\{|\}|~|\^|/|\'|\"|°|º", " ", sent) for sent in data]

    # remove saudação
    data = [remover_saudacao(sent) for sent in data]
    
    # remove saudação
    data = [remover_empresa_nome(sent) for sent in data]
    
    # remove regionalismos
    data = [remover_regionalismo(sent) for sent in data]
    
    # remove tags
    data = [re.sub('\<script.*script>', '', str(sent)) for sent in data]
    
#     remover tag 
    data = [re.sub('\<a.*a>', '', str(sent)) for sent in data]
    
    # remover repetições como "oiiiiiii"
    data = [remover_repeticao_de_letras(sent) for sent in data]
    
#     data = [Stemming(sent) for sent in data]
    
    return data

print("Métodos para o pré-processamento carregados!")

Métodos para o pré-processamento carregados!


# 4. Ajuste nos dados carregados 

## 4.1 Separar coluna "created" 
Nos dados das reclamações é necessário dividir a coluna "created" em outros duas colunas, chamadas de data e hora.

### 4.1.1 Executando a separação e criando as duas novas colunas 

In [9]:
# df_americanas_com_loja_online.created

In [10]:
# date, time_created = separa_coluna(list(df_americanas_com_loja_online.created),"T")
# df_americanas_com_loja_online['date'] = date
# df_americanas_com_loja_online['time_created'] = time_created

# date, time_created = separa_coluna(list(df_magazine_luiza_loja_online.created),"T")
# df_magazine_luiza_loja_online['date'] = date
# df_magazine_luiza_loja_online['time_created'] = time_created

# date, time_created = separa_coluna(list(df_mercado_livre.created),"T")
# df_mercado_livre['date'] = date
# df_mercado_livre['time_created'] = time_created

# date, time_created = separa_coluna(list(df_netshoes.created),"T")
# df_netshoes['date'] = date
# df_netshoes['time_created'] = time_created

# date, time_created = separa_coluna(list(df_submarino.created),"T")
# df_submarino['date'] = date
# df_submarino['time_created'] = time_created

# print("Criação das colunas para data e para a hora foi finalizada!")

In [11]:
## Separar coluna "city_state" em cidades e estados (consumidor.gov)

In [12]:
cidade, estado = separa_coluna(list(df_americanas_com_loja_online.local)," - ")
df_americanas_com_loja_online['cidade'] = cidade
df_americanas_com_loja_online['estado'] = estado

cidade, estado = separa_coluna(list(df_magazine_luiza_loja_online.local)," - ")
df_magazine_luiza_loja_online['cidade'] = cidade
df_magazine_luiza_loja_online['estado'] = estado

cidade, estado = separa_coluna(list(df_mercado_livre.local)," - ")
df_mercado_livre['cidade'] = cidade
df_mercado_livre['estado'] = estado

cidade, estado = separa_coluna(list(df_netshoes.local)," - ")
df_netshoes['cidade'] = cidade
df_netshoes['estado'] = estado

cidade, estado = separa_coluna(list(df_submarino.local)," - ")
df_submarino['cidade'] = cidade
df_submarino['estado'] = estado

print("Criação das colunas para a cidade e para o estado foi finalizada")

Criação das colunas para a cidade e para o estado foi finalizada


## 4.2 Remover linha nos dataframes das empresas com dados incorretos na coluna "state"

In [13]:
# df_mercado_livre.userState

In [14]:
ruidos = ['*','None','G','C', 'B','E', 'P','--','-', 'M', 'R', 'S', 'naoconsta']

for ruido in ruidos:
    df_americanas_com_loja_online = df_americanas_com_loja_online[df_americanas_com_loja_online.estado !=ruido]
    df_magazine_luiza_loja_online = df_magazine_luiza_loja_online[df_magazine_luiza_loja_online.estado != ruido]
    df_mercado_livre = df_mercado_livre[df_mercado_livre.estado != ruido]
    df_netshoes = df_netshoes[df_netshoes.estado != ruido]
    df_submarino = df_submarino[df_submarino.estado != ruido]
print("Remoção dos ruidos doi finalizada!")

Remoção dos ruidos doi finalizada!


In [15]:
df_mercado_livre.data[1]

' 01/02/2019'

In [16]:
# import copy
# df_tim_antes = copy.copy(df_tim)
# df_vivo_antes = copy.copy(df_vivo)
# df_oi_antes = copy.copy(df_oi)
# df_claro_antes = copy.copy(df_claro)

# 6. Préprocessamento 

In [17]:
# df_americanas_com_loja_online

In [18]:
# df_americanas_com_loja_online.description = processar_dados(list(df_americanas_com_loja_online.description))
# df_magazine_luiza_loja_online.description = processar_dados(list(df_magazine_luiza_loja_online.description))
# df_mercado_livre.description = processar_dados(list(df_mercado_livre.description))
# df_netshoes.description = processar_dados(list(df_netshoes.description))
# df_submarino.description = processar_dados(list(df_submarino.description))

In [19]:
df_americanas_com_loja_online.relato = processar_dados(list(df_americanas_com_loja_online.relato))
df_magazine_luiza_loja_online.relato = processar_dados(list(df_magazine_luiza_loja_online.relato))
df_mercado_livre.relato = processar_dados(list(df_mercado_livre.relato))
df_netshoes.relato = processar_dados(list(df_netshoes.relato))
df_submarino.relato = processar_dados(list(df_submarino.relato))

In [20]:
# df_tim.description

In [21]:
# print(df_tim.description[2])
# print(df_tim_antes.description[2])

In [22]:
df_americanas_com_loja_online.to_csv("americanas_pre_cgov.csv", sep='\t', index=False)
df_magazine_luiza_loja_online.to_csv("magazine_luiza_pre_cgov.csv", sep='\t', index=False)
df_mercado_livre.to_csv("mercado_livre_pre_cgov.csv", sep='\t', index=False)
df_netshoes.to_csv("netshoes_pre_cgov.csv", sep='\t', index=False)
df_submarino.to_csv("submarino_pre_cgov.csv", sep='\t', index=False)